In [1]:
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [2]:
import folium

from main import MapItems, Locator, MapMaker

In [3]:
locator = Locator()

In [57]:
mm = MapMaker(map_item_filename='london_map_items.json')
mm.main()

In [58]:
mm.base_map

In [6]:
locator("Platform 9 3/4", "London United Kingdom")

Getting location for Platform 9 3/4 London United Kingdom...
Success!


[51.5321878, -0.12390657687861176]

# Tube

In [7]:
import pandas as pd

In [8]:
df_stations = pd.read_csv("London stations.csv")

In [9]:
df_lines = pd.read_csv("London tube lines.csv")

In [10]:
df_stations.head()

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508757,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516886,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN


In [11]:
df_lines.head()

,Tube Line,From Station,To Station
0,Bakerloo,Baker Street,Regents Park
1,Bakerloo,Charing Cross,Embankment
2,Bakerloo,Edgware Road (Bakerloo),Marylebone
3,Bakerloo,Embankment,Waterloo
4,Bakerloo,Harlesden,Willesden Junction


In [11]:
df = df_stations.loc[:, ['Station', "Latitude", "Longitude", "Zone"]].copy()

In [27]:
tube_lines = pd.concat([
    df_lines.set_index("From Station")['Tube Line'],
    df_lines.set_index("To Station")['Tube Line']
])
tube_lines = tube_lines.loc[~tube_lines.index.duplicated()]
tube_lines.head()

Baker Street               Bakerloo
Charing Cross              Bakerloo
Edgware Road (Bakerloo)    Bakerloo
Embankment                 Bakerloo
Harlesden                  Bakerloo
Name: Tube Line, dtype: object

In [13]:
df = df.set_index('Station')
df['TubeLine'] = tube_lines[df.index]
df = df.reset_index(drop=False)

In [14]:
df.head()

,Station,Latitude,Longitude,Zone,TubeLine
0,Abbey Road,51.531952,0.003723,3,DLR
1,Abbey Wood,51.490784,0.120272,4,Southeastern
2,Acton Central,51.508757,-0.263430,2,Overground
3,Acton Main Line,51.516886,-0.267690,3,Elizabeth
4,Acton Town,51.503071,-0.280303,3,District


In [15]:
df.loc[df['Zone'] == '1', 'TubeLine'].unique()

array(['Circle', 'District', 'Northern', 'Bakerloo', 'Central',
       'Thameslink', 'Piccadilly', 'London Midland', 'C2C', 'Jubilee',
       'Great Northern', 'Victoria', 'Overground', 'DLR', 'Southeastern'],
      dtype=object)

In [16]:
df["Zone"].unique()

array(['3', '4', '2', '3,4,5,6', '5', '1', '9', '2,3', '6', '3,4', '7',
       nan, '8', '1,2', '5,6', '6,7'], dtype=object)

In [17]:
df_lines.loc[df_lines['Tube Line'] == 'Bakerloo', :]

,Tube Line,From Station,To Station
0,Bakerloo,Baker Street,Regents Park
1,Bakerloo,Charing Cross,Embankment
2,Bakerloo,Edgware Road (Bakerloo),Marylebone
3,Bakerloo,Embankment,Waterloo
4,Bakerloo,Harlesden,Willesden Junction
5,Bakerloo,Harrow and Wealdstone,Kenton
6,Bakerloo,Kensal Green,Queens Park
7,Bakerloo,Kenton,South Kenton
8,Bakerloo,Kilburn Park,Maida Vale
9,Bakerloo,Lambeth North,Elephant and Castle


In [31]:
# _from = pd.Index(df_lines['From Station'])
# _to = pd.Index(df_lines['To Station'])
# start_stations = _from.difference(_to)
# end_stations = _to.difference(_from)
# intermediate_stations = _from.intersection(_to)
# no_end_stations = _from.union(_to).difference(end_stations)

In [52]:
df.head()

,Latitude,Longitude,Zone,TubeLine
Station,,,,
Abbey Road,51.531952,0.003723,3,DLR
Abbey Wood,51.490784,0.120272,4,Southeastern
Acton Central,51.508757,-0.263430,2,Overground
Acton Main Line,51.516886,-0.267690,3,Elizabeth
Acton Town,51.503071,-0.280303,3,District


In [ ]:
df['Zone'].fillna('')

In [53]:
d = dict()

In [54]:
d['Stations'] = dict()

In [63]:
for station, row in df.iterrows():
    d['Stations'][station] = {
        "coordinates": [row['Latitude'], row['Longitude']],
        "Zone": row['Zone'],
        'TubeLine': row['TubeLine']
    }

In [ ]:
list(zip(df_lines['From Station'], df_lines['To Station']))

In [100]:
df_gb = df_lines.groupby("Tube Line").apply(lambda x: dict(zip(x['From Station'], x['To Station'])))

In [113]:
d["TubeLines"] = dict()
for line, val in df_gb.to_frame().iterrows():
    d['TubeLines'][line] = val[0]

In [12]:
import json

In [115]:
def to_json(filename, data):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4, sort_keys=True)

In [118]:
to_json("tube.json", d)

In [14]:
def from_json(filename: str):
    """Loads a json file"""
    with open(filename, 'r') as json_file:
        data = json.load(json_file)
    return data

In [15]:
tube = from_json("tube.json")

In [29]:
bakerloo = tube["TubeLines"]['Bakerloo']
bakerloo

{'Baker Street': 'Regents Park',
 'Charing Cross': 'Embankment',
 'Edgware Road (Bakerloo)': 'Marylebone',
 'Embankment': 'Waterloo',
 'Harlesden': 'Willesden Junction',
 'Harrow and Wealdstone': 'Kenton',
 'Kensal Green': 'Queens Park',
 'Kenton': 'South Kenton',
 'Kilburn Park': 'Maida Vale',
 'Lambeth North': 'Elephant and Castle',
 'Maida Vale': 'Warwick Avenue',
 'Marylebone': 'Baker Street',
 'North Wembley': 'Wembley Central',
 'Oxford Circus': 'Piccadilly Circus',
 'Paddington': 'Edgware Road (Bakerloo)',
 'Piccadilly Circus': 'Charing Cross',
 'Queens Park': 'Kilburn Park',
 'Regents Park': 'Oxford Circus',
 'South Kenton': 'North Wembley',
 'Stonebridge Park': 'Harlesden',
 'Warwick Avenue': 'Paddington',
 'Waterloo': 'Lambeth North',
 'Wembley Central': 'Stonebridge Park',
 'Willesden Junction': 'Kensal Green'}

In [94]:
from folium import plugins

In [148]:
mm = MapMaker(map_item_filename='london_map_items.json')
mm.main()

In [149]:
fg_all = folium.FeatureGroup('Tube lines')
mm.base_map.add_child(fg_all)

fg_tube_lines = []
for tube_line, stations in tube['TubeLines'].items():
    fg = plugins.FeatureGroupSubGroup(fg_all, tube_line)
    mm.base_map.add_child(fg)
    color = colors.get(tube_line, '#000000')

    for start, end in stations.items():
        start_loc = tube['Stations'][start]['coordinates']
        end_loc = tube['Stations'][end]['coordinates']
        folium.PolyLine([start_loc, end_loc], color=f'{color}', weight=5, opacity=0.7).add_to(fg)
    
    fg_tube_lines.append(fg)


In [160]:
fg_zones_all = folium.FeatureGroup('Tube zones', control=False)
mm.base_map.add_child(fg_zones_all)

fg_zones = {f"Zone {i}": plugins.FeatureGroupSubGroup(fg_zones_all, f"Zone{i}") for i in range(1, 10)}
fg_zones.update({"None": plugins.FeatureGroupSubGroup(fg_zones_all, "None")})

for station, val in tube['Stations'].items():
    zone = val['Zone']
    if zone:
        
    else:
        

In [150]:
folium.LayerControl(collapsed=False).add_to(mm.base_map)
plugins.GroupedLayerControl(
    groups={'Tube lines': fg_tube_lines},
    exclusive_groups=False,
    collapsed=False,
).add_to(mm.base_map)

In [151]:
mm.base_map

In [169]:
plugins.TagFilterButton?

Init signature:
plugins.TagFilterButton(
    data,
    icon='fa-filter',
    clear_text='clear',
    filter_on_every_click=True,
    open_popup_on_hover=False,
    **kwargs,
)
Docstring:     
Creates a Tag Filter Button to filter markers based on criteria
(https://github.com/maydemirx/leaflet-tag-filter-button)

Parameters
----------
data: list, of strings.
    The tags to filter for this filter button.
icon: string, default 'fa-filter'
    The icon for the filter button
clear_text: string, default 'clear'
    Text of the clear button
filter_on_every_click: bool, default True
    if True, the plugin will filter on every click event on checkbox.
open_popup_on_hover: bool, default False
    if True, popup that contains tags will be open at mouse hover time
File:           c:\mambaforge\envs\main\lib\site-packages\folium\plugins\tag_filter_button.py
Type:           type
Subclasses:     

In [50]:
colors = {
    "Bakerloo": [179, 99, 5],
    "Central": [227, 32, 23],
    "Circle": [255, 211, 0],
    "District": [0, 120, 42],
    "Elizabeth": [105, 80, 161],
    "Hammersmith and City": [243, 169, 187],
    "Jubilee": [160, 165, 169],
    "Metropolitan": [155, 0, 86],
    "Northern": [0, 0, 0],
    "Piccadilly": [0, 54, 136],
    "Victoria": [0, 152, 212],
    "Waterloo & City": [149, 205, 186],
    "DLR": [0, 164, 167],
    "London Overground": [238, 124, 14],
    "London Trams": [132, 184, 23],
    "Emirates Cable Car": [226, 24, 54]
}

In [59]:
colors = {
    "Bakerloo": "#B36305",
    "Central": "#E32017",
    "Circle": "#FFD300",
    "District": "#00782A",
    "Hammersmith and City": "#F3A9BB",
    "Jubilee": "#A0A5A9",
    "Metropolitan": "#9B0056",
    "Northern": "#000000",
    "Piccadilly": "#003688",
    "Victoria": "#0098D4",
    "Waterloo and City": "#95CDBA",
    "DLR": "#00A4A7",
    "Overground": "#EE7C0E",
    "Tramlink": "#84B817",
    "Cable Car": "#E21836 	",
    "Crossrail": "#7156A5",
}

In [51]:
list(tube['TubeLines'].keys())

['Bakerloo',
 'C2C',
 'Central',
 'Chiltern Railways',
 'Circle',
 'DLR',
 'District',
 'Elizabeth',
 'Great Northern',
 'Great Western',
 'Greater Anglia',
 'Hammersmith and City',
 'Heathrow Connect',
 'Heathrow Express',
 'Jubilee',
 'London Midland',
 'Metropolitan',
 'Northern',
 'Overground',
 'Piccadilly',
 'South Western',
 'Southeastern',
 'Southern',
 'Thameslink',
 'Tramlink',
 'Victoria',
 'Waterloo and City']

In [154]:
import numpy as np

array(['', '1', '1,2', '2', '2,3', '3', '3,4', '3,4,5,6', '4', '5', '5,6',
       '6', '6,7', '7', '8', '9'], dtype='<U7')

In [168]:
m = folium.Map(location=[0, 0], zoom_start=6)

fg = folium.FeatureGroup(name="groups")
m.add_child(fg)

g1 = plugins.FeatureGroupSubGroup(fg, "group1")
m.add_child(g1)

g_hor = plugins.FeatureGroupSubGroup(fg, "hor")
m.add_child(g_hor)
g_ver = plugins.FeatureGroupSubGroup(fg, "ver")
m.add_child(g_ver)

g2 = plugins.FeatureGroupSubGroup(fg, "group2")
m.add_child(g2)

folium.Marker([-1, -1]).add_to(g1)
# folium.Marker([1, 1]).add_to(g1)
marker = folium.Marker([1, 1])
g1.add_child(marker)
# g11.add_child(marker)

folium.Marker([-1, 1]).add_to(g2)
folium.Marker([1, -1]).add_to(g2)

folium.LayerControl(collapsed=False).add_to(m)

m

In [174]:
TagFilterButton?

Init signature:
TagFilterButton(
    data,
    icon='fa-filter',
    clear_text='clear',
    filter_on_every_click=True,
    open_popup_on_hover=False,
    **kwargs,
)
Docstring:     
Creates a Tag Filter Button to filter markers based on criteria
(https://github.com/maydemirx/leaflet-tag-filter-button)

Parameters
----------
data: list, of strings.
    The tags to filter for this filter button.
icon: string, default 'fa-filter'
    The icon for the filter button
clear_text: string, default 'clear'
    Text of the clear button
filter_on_every_click: bool, default True
    if True, the plugin will filter on every click event on checkbox.
open_popup_on_hover: bool, default False
    if True, popup that contains tags will be open at mouse hover time
File:           c:\mambaforge\envs\main\lib\site-packages\folium\plugins\tag_filter_button.py
Type:           type
Subclasses:     

In [172]:
import numpy as np
import random

# Generate base data
data = (np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
                np.array([[48, 5]]))
# Generate the data to segment by (levels of another pandas column in practical usage)
categories = ['category{}'.format(i+1) for i in range(5)]
category_column = [random.choice(categories) for i in range(len(data))]

In [175]:
from folium.plugins import TagFilterButton

# Create map and add the data with additional parameter tags as the segmentation
m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=7)
for i, latlng in enumerate(data):
    category = category_column[i]
    folium.Marker(
        tuple(latlng),
        tags=[category]
    ).add_to(m)
    
TagFilterButton(categories).add_to(m)
TagFilterButton(['a', 'b']).add_to(m)


m